# Optical character recognition with an HHL support vector machine

## The idea

We want a full HHL-based support vector machine (SVM), which can identify certain characters. To keep this algorithm relatively easy to build, we will restrict ourselves to a SVM wich can decide whether a small binary (pure black and white) image displays the character '6' or '9'. At first, the image has to be proprocessed to extract a number of features. We will use the ratio between the number of black pixels between the left (upper) and right (lower) half of the image as our two features. With two training samples of '6' and '9' a SVM can be built which can classify any other written 6 or 9. We begin with the basic concept of the SVM.

## The (linear) support vector machine
A support vecotr machine can classify input data into two groups. This training data is provided with the correspoding label of the group, hence the SVM is an instance of a supervised machine learning algorithm. The key idea is to find a hyperplane (for two feature dimensions, that is a line) that divides the feature space into two sections. 
<img src="https://upload.wikimedia.org/wikipedia/commons/b/b5/Svm_separating_hyperplanes_%28SVG%29.svg" width="400px"/>

### A bit of math
Suppose you have training data set
$$ (\vec{x_1}, y_1), (\vec{x_2}, y_2), \dots, (\vec{x_n}, y_n) $$ 
where $\vec{x_i}$ is the vector in feature space and $y_i$ are either $1$ or $-1$ indicating the corresponding group.
The goal is now to find a hyperplane, described with the normal-vector $\vec{w}$ and an offset parameter $b$, that has the largest distance to any of the input points. This plane is therefore constructed by minimizing $|\vec{w}|^2/2$ under the constraint $y_i (\vec{x_i}\cdot\vec{w}+b) \geq 1$.
One can reformulate given optimalization problem with so called Kuhn-Tucker multipliers $\vec{\alpha}=(\alpha_1, \dots, \alpha_n)^T$
$$ L(\vec{\alpha}) = \sum_j^n y_j\alpha_j-\frac{1}{2}\sum_{j,k=1}^n\alpha_jK_{jk}\alpha_k$$
with $K_{jk}=\vec{x_j}\cdot\vec{x_k}$ and the constraints
$$ \sum_{j=1}^n\alpha_j = 0, \quad y_j\alpha_j\geq0. $$
Therefore the parameters can be recovered with
$$ \vec{w} = \sum_{j=1}^n\alpha_j\vec{x_j}, \quad b = y_j-\vec{w}\cdot\vec{x_j} \,\,\,\text{if}\,\, \alpha_j \neq 0$$
only a small set of $\alpha_j$s are non-zero and should give the same result.

To classify a new input vector, just the $\alpha_j$s are neccessary
$$ y(\vec{x}) = \text{sign}\left(\sum_{j=1}^n  \alpha_j \vec{x_j}\cdot\vec{x}+b\right). $$

A least square approximation of the upper minimization problem leads to following linear equation
$$ F \begin{pmatrix}b\\\vec{\alpha}\end{pmatrix} = 
 \begin{bmatrix} 0 & \vec{1}^T \\\vec{1} & K+\gamma^{-1} \mathbb{1}\end{bmatrix} \begin{pmatrix}b\\\vec{\alpha}\end{pmatrix} = \begin{pmatrix}0\\\vec{y}\end{pmatrix}, $$
 whose result will be the optimized parameters for classifying new inputs. $\gamma$ is a user-specified parameter which determines the relative weight of the training error.

# The quantum support vector machine

We have already seen that the HHL algorithm brings big advantage to problem of solving linear systems of equations. However, the output of the HHL is fully encoded in a quantum state, in our case
$$ |b, \vec{\alpha}\rangle = \frac{1}{\sqrt{C}}\left(b|0\rangle+\sum_{j=1}^n\alpha_j|j\rangle\right),$$
but we cannot extract this information easily. However, it is possible to construct a query state for some data and calculate the classification on the quantum computer. 

For that, we need a black box operation $U$ which is able to produce the state
$$ \frac{1}{\sqrt{n}} \sum_j^n |j\rangle|0\rangle \overset{U}{\rightarrow} \frac{1}{\sqrt{N}} \sum_{j=1}^n \left|\vec{x_j}\right|\,|j\rangle|\vec{x_j}\rangle. $$
Applying $U$ to $|b, \alpha\rangle$ gives the state 
$$ |\tilde{u}\rangle = \frac{1}{\sqrt{N_\tilde{u}}}\left(b|0\rangle|0\rangle+\sum_{j=1}^n \alpha_j\left|\vec{x_j}\right|\,|j\rangle|\vec{x_j}\rangle\right). $$

We also need another black box $V$ which prepares the query vector $\vec{x}$
$$ |\tilde{x}\rangle = \frac{1}{\sqrt{N_\tilde{x}}}\left(|0\rangle|0\rangle+\sum_{j=1}^n \alpha_j\left|\vec{x}\right|\,|j\rangle|\vec{x}\rangle\right). $$

And then we can calculate the dot product of $\tilde{x}$ and $\tilde{u}$
$$ \langle\tilde{x}|\tilde{u}\rangle \propto b + \sum_{j=1}^n \left|\vec{x}\right| \left|\vec{x_j}\right| \langle\vec{x}|\vec{x_j} \rangle  = b + \sum_{j=1}^n \alpha_j\vec{x}\cdot\vec{x_j}. $$ So the sign of this product gives the class of the query vector.

The dot product can be calculated by a so-called swap test. Let $U$ and $V$ be controllable, then we can produce the state
$$ \frac{1}{\sqrt{2}}(|0\rangle|\tilde{u}\rangle + |1\rangle|\tilde{x}\rangle). $$
Applying the Hadamard gate on the first qubit and measuring it gives following probabilities for obtaining the state $|0\rangle$
$$ P_0 = \frac{1}{2}(1+\langle\tilde{x}|\tilde{u}\rangle). $$
So if more than half of the measurements result in $|0\rangle$, The sign of $\langle\tilde{x}|\tilde{u}\rangle$ must be positive, therefore $\vec{x}$ belongs to the group with $y_i = 1$.  

The Complete circuit looks like this